In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!pip3 install kmodes
!pip install catboost
!pip install pingouin
!pip install scipy

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np
import pingouin as pg
import scipy.stats as stats
pd.set_option('display.max_columns',None)
pd.set_option('display.max_row',None)
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler


# 1.집단 0의 녹조 예측 : 변수 중요도 & 회귀계수

### 집단 0 후진제거법

In [ ]:
def backward_regression(X, y,initial_list=[], threshold_out = 0.05,  feature_list = df0[final_reasoncol].drop(columns=['녹조']).columns.tolist()):
    sv_per_step = [] ## 각 스텝별로 선택된 변수들
    adj_r_squared_list = [] ## 각 스텝별 수정된 결정계수
    steps = [] ## 스텝
    step = 0
    included = feature_list
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0)
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:] # 각 feature의 P값을 의미함
        worst_pval = pvalues.max()	# P 값이 가장 높은 것 선정
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)

        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0).rsquared_adj
        adj_r_squared_list.append(adj_r_squared)
        sv_per_step.append(included.copy())

        if not changed:
            break

    return included,step,steps,adj_r_squared_list,sv_per_step

x_data0 = df0[final_reasoncol].drop(columns=['녹조'])
target0 = df0[['녹조']]
backward_valriables_function0,step,steps0,adj_r_squared_list0,sv_per_step0 = backward_regression(x_data0, target0)

In [ ]:
# 스탭별로 변수가 제거되는 과정을 저장한 리스트 : sv_per_step
print(len(sv_per_step0))
print(len(sv_per_step0[0]))

In [ ]:
# 변수가 11개만 선택했을 때의 인덱스를 확인
for i in range(19):
  if len(sv_per_step0[i]) == 11:
    print(i)

In [ ]:
# 최종으로 선택된 11개 확인
print(sv_per_step0[9],end="")

### 집단 0 랜덤포레스트

In [ ]:
sns.heatmap(x_data0[['BOD(㎎/L)_x', 'COD(㎎/L)_x', 'I-131(요오드)', 'DO(㎎/L)',  'TN(㎎/L)',
                     'TOC(㎎/L)', '방류_BOD', '방류_TP', '총_유동인구_수_상구', '한강옆녹지_하구']].corr(), annot = True, fmt = '.3f', cmap = 'RdYlBu_r',  vmin = -1, vmax = 1)
plt.show()

In [ ]:
X_train0, X_test0, y_train0, y_test0 = train_test_split(x_data0[['BOD(㎎/L)_x', 'COD(㎎/L)_x', 'I-131(요오드)', 'DO(㎎/L)',  'TN(㎎/L)',
                                                                 'TOC(㎎/L)', '방류_BOD', '방류_TP', '총_유동인구_수_상구', '한강옆녹지_하구']], target0, random_state=42) # 열을 바꾸고 싶으면 sv_per_step[8] 부분 바꾸면 됨

params = {
    'n_estimators':(100, 200),
    'max_depth' : (5, 10),
    'min_samples_leaf' : (5, 18),
    'min_samples_split' : (5, 16)
}
rf_run0 = RandomForestRegressor(random_state=0, n_jobs=-1)
grid_cv0 = GridSearchCV(rf_run0, param_grid=params, cv=2, n_jobs=-1)
grid_cv0.fit(X_train0, y_train0)
print('최적 하이퍼 파라미터:', grid_cv0.best_params_)
print('최적 예측 정확도: {0:.4f}'.format(grid_cv0.best_score_))

In [ ]:
rf_run0 = RandomForestRegressor(random_state=0, max_depth=10, min_samples_leaf=5, min_samples_split=5,n_estimators=200)
rf_run0.fit(X_train0, y_train0)

train_predict0 = rf_run0.predict(X_train0)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict0, y_train0))) ) # 0.09822104783705203

# validation rmse
valid_predict0 = rf_run0.predict(X_test0)
print("RMSE':{}".format(math.sqrt(mean_squared_error(valid_predict0, y_test0))) ) # 0.02920048677256981

In [ ]:
feature_importance0 = rf_run0.feature_importances_
print("Feature Importance:")
col_name0 = []
feature_score0 = []
for feature, importance in zip(X_train0.columns, feature_importance0):
  col_name0.append(feature)
  feature_score0.append(importance)

feature_df0 = pd.DataFrame({'열':col_name0,'feature_importance':feature_score0})
feature_df0.sort_values(by='feature_importance', ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False
sorted_indices = np.argsort(feature_importance0)[::-1]
sorted_importance = feature_importance0[sorted_indices]
sorted_variables = np.array(X_train0.columns)[sorted_indices]

# 가로막대 그래프 생성
plt.figure(figsize=(8, 6))
plt.barh(sorted_variables, sorted_importance, color='skyblue')
plt.xlabel('중요도')
plt.ylabel('변수 이름')
plt.title('변수 중요도')


### 집단 0 회귀계수

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터 불러오기


# for b0, 상수항 추가
x_data01 = sm.add_constant(x_data0[['BOD(㎎/L)_x', 'COD(㎎/L)_x', 'I-131(요오드)', 'DO(㎎/L)',  'TN(㎎/L)',
                                                                 'TOC(㎎/L)', '방류_BOD', '방류_TP', '총_유동인구_수_상구', '한강옆녹지_하구']], has_constant = "add") # 열을 바꾸고 싶다면 sv_per_step[8]이 부분을 바꾸면 됨

# OLS 검정
multi_model0 = sm.OLS(target0, x_data01)
fitted_multi_model0 = multi_model0.fit()
print(fitted_multi_model0.summary())
col_list0 = []
val_list0 = []
variable_names0 = list(fitted_multi_model0.params.index)
coefficients0 = list(fitted_multi_model0.params)
for i, coef in enumerate(coefficients0):
    variable_name0 = variable_names0[i]
    col_list0.append(variable_name0)
    val_list0.append(coef)

coef_df0 = pd.DataFrame({'열':col_list0,'회귀계수':val_list0})
coef_df0.sort_values('회귀계수',ascending=False)


### 집단0 shap 해보기

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False
import shap
explainer = shap.TreeExplainer(rf_run0)
shap_values = explainer.shap_values(X_test0)
shap.initjs() # javascript 초기화 (graph 초기화)
shap.force_plot(explainer.expected_value, shap_values[1,:], X_test0.iloc[1,:])
shap.summary_plot ( shap_values , X_test0 )

# 2.집단 1의 녹조 예측 : 변수 중요도 & 회귀계수

### 집단 1 후진제거법

In [ ]:
def backward_regression(X, y,initial_list=[], threshold_out = 0.05,  feature_list = df1[final_reasoncol].drop(columns=['녹조']).columns.tolist()):
    sv_per_step = [] ## 각 스텝별로 선택된 변수들
    adj_r_squared_list = [] ## 각 스텝별 수정된 결정계수
    steps = [] ## 스텝
    step = 0
    included = feature_list
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0)
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:] # 각 feature의 P값을 의미함
        worst_pval = pvalues.max()	# P 값이 가장 높은 것 선정
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)

        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0).rsquared_adj
        adj_r_squared_list.append(adj_r_squared)
        sv_per_step.append(included.copy())

        if not changed:
            break

    return included,step,steps,adj_r_squared_list,sv_per_step

x_data1 = df1[final_reasoncol].drop(columns=['녹조'])
target1 = df1[['녹조']]
backward_valriables_function1,step,steps1,adj_r_squared_list1,sv_per_step1 = backward_regression(x_data1, target1)

In [ ]:
# 스탭별로 변수가 제거되는 과정을 저장한 리스트 : sv_per_step
print(len(sv_per_step1))
print(len(sv_per_step1[0]))

In [ ]:
# 최종으로 선택된 11개 확인
sv_per_step1[10]

In [ ]:
sns.heatmap(x_data0[['COD(㎎/L)_x',
 'DO(㎎/L)',
 'TN(㎎/L)',
 '하동분기별인구수',
 '2차하수처리량',
 '교통량',
 '총_유동인구_수_상구',
 '총_유동인구_수_하구',
 '한강옆녹지_하구',
 '시민의식_환경']].corr(), annot = True, fmt = '.3f', cmap = 'RdYlBu_r',  vmin = -1, vmax = 1)
plt.show()

### 집단 1 랜덤포레스트

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(x_data1[sv_per_step1[10]], target1, random_state=42) # 열을 바꾸고 싶으면 sv_per_step[8] 부분 바꾸면 됨

params = {
    'n_estimators':(100, 200),
    'max_depth' : (5, 10),
    'min_samples_leaf' : (5, 18),
    'min_samples_split' : (8, 16)
}
rf_run1 = RandomForestRegressor(random_state=0, n_jobs=-1)
grid_cv1 = GridSearchCV(rf_run1, param_grid=params, cv=2, n_jobs=-1)
grid_cv1.fit(X_train1, y_train1)
print('최적 하이퍼 파라미터:', grid_cv1.best_params_)
print('최적 예측 정확도: {0:.4f}'.format(grid_cv1.best_score_))

In [ ]:
rf_run1 = RandomForestRegressor(random_state=0, max_depth=5, min_samples_leaf=5, min_samples_split=8,n_estimators=200)
rf_run1.fit(X_train1, y_train1)

train_predict1 = rf_run1.predict(X_train1)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict1, y_train1))) ) # 0.09822104783705203

# validation rmse
valid_predict1 = rf_run1.predict(X_test1)
print("RMSE':{}".format(math.sqrt(mean_squared_error(valid_predict1, y_test1))) ) # 0.02920048677256981

In [ ]:
feature_importance1 = rf_run1.feature_importances_
print("Feature Importance:")
col_name1 = []
feature_score1 = []
for feature, importance in zip(X_train1.columns, feature_importance1):
  col_name1.append(feature)
  feature_score1.append(importance)

feature_df1 = pd.DataFrame({'열':col_name1,'feature_importance':feature_score1})
feature_df1.sort_values(by='feature_importance', ascending=False)

In [ ]:
sorted_indices = np.argsort(feature_importance1)[::-1]
sorted_importance = feature_importance1[sorted_indices]
sorted_variables = np.array(X_train1.columns)[sorted_indices]

# 가로막대 그래프 생성
plt.figure(figsize=(8, 6))
plt.barh(sorted_variables, sorted_importance, color='skyblue')
plt.xlabel('중요도')
plt.ylabel('변수 이름')
plt.title('변수 중요도')


### 집단1 회귀계수

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
pd.set_option('display.float_format', '{:.7f}'.format)

# 데이터 불러오기


# for b0, 상수항 추가
x_data11 = sm.add_constant(x_data1[sv_per_step1[10]], has_constant = "add") # 열을 바꾸고 싶다면 sv_per_step[8]이 부분을 바꾸면 됨

# OLS 검정
multi_model1 = sm.OLS(target1, x_data11)
fitted_multi_model1 = multi_model1.fit()
print(fitted_multi_model1.summary())
col_list1 = []
val_list1 = []
variable_names1 = list(fitted_multi_model1.params.index)
coefficients1 = list(fitted_multi_model1.params)
for i, coef in enumerate(coefficients1):
    variable_name = variable_names1[i]
    col_list1.append(variable_name)
    val_list1.append(coef)

coef_df1 = pd.DataFrame({'열':col_list1,'회귀계수':val_list1})
coef_df1['회귀계수'] =coef_df1['회귀계수'].astype(float)
coef_df1.sort_values('회귀계수',ascending=False)


In [ ]:
explainer = shap.TreeExplainer(rf_run1)
shap_values = explainer.shap_values(X_test1)
shap.initjs() # javascript 초기화 (graph 초기화)
shap.force_plot(explainer.expected_value, shap_values[1,:], X_test1.iloc[1,:])
shap.summary_plot ( shap_values , X_test1 )

# 3.집단 2의 녹조 예측 : 변수 중요도 & 회귀계수

### 집단 2 후진제거법

In [ ]:
def backward_regression(X, y,initial_list=[], threshold_out = 0.05,  feature_list = df2[final_reasoncol].drop(columns=['녹조']).columns.tolist()):
    sv_per_step = [] ## 각 스텝별로 선택된 변수들
    adj_r_squared_list = [] ## 각 스텝별 수정된 결정계수
    steps = [] ## 스텝
    step = 0
    included = feature_list
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0)
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:] # 각 feature의 P값을 의미함
        worst_pval = pvalues.max()	# P 값이 가장 높은 것 선정
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)

        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0).rsquared_adj
        adj_r_squared_list.append(adj_r_squared)
        sv_per_step.append(included.copy())

        if not changed:
            break

    return included,step,steps,adj_r_squared_list,sv_per_step

x_data2 = df2[final_reasoncol].drop(columns=['녹조'])
target2 = df2[['녹조']]
backward_valriables_function2,step,steps2,adj_r_squared_list2,sv_per_step2 = backward_regression(x_data2, target2)

In [ ]:
# 최종으로 선택된 11개 확인
print(sv_per_step2[10],end="") #15는 되어야지 시민의식환경 사라짐

In [ ]:
sns.heatmap(x_data0[['COD(㎎/L)_x', '총질소(T-N)(㎎/L)', 'I-131(요오드)', 'DO(㎎/L)', 'TOC(㎎/L)', '하동분기별인구수', '상동분기별인구수', '방류_BOD', '한강옆녹지_하구', '시민의식_환경']].corr(), annot = True, fmt = '.3f', cmap = 'RdYlBu_r',  vmin = -1, vmax = 1)
plt.show()

In [ ]:
x_data2 = df2[sv_per_step2[10]]
target2 = df2[['녹조']]

### 집단 2 랜덤포레스트

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(x_data2, target2, random_state=42) # 열을 바꾸고 싶으면 sv_per_step[8] 부분 바꾸면 됨

params = {
    'n_estimators':(100, 200),
    'max_depth' : (5, 10),
    'min_samples_leaf' : (5, 18),
    'min_samples_split' : (5, 16)
}
rf_run2 = RandomForestRegressor(random_state=0, n_jobs=-1)
grid_cv2 = GridSearchCV(rf_run2, param_grid=params, cv=2, n_jobs=-1)
grid_cv2.fit(X_train2, y_train2)
print('최적 하이퍼 파라미터:', grid_cv2.best_params_)
print('최적 예측 정확도: {0:.4f}'.format(grid_cv2.best_score_))

In [ ]:
rf_run2 = RandomForestRegressor(random_state=0, max_depth=5, min_samples_leaf=18, min_samples_split=5,n_estimators=100)
rf_run2.fit(X_train2, y_train2)

train_predict2 = rf_run2.predict(X_train2)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict2, y_train2))) ) # 0.09822104783705203

# validation rmse
valid_predict2 = rf_run2.predict(X_test2)
print("RMSE':{}".format(math.sqrt(mean_squared_error(valid_predict2, y_test2))) ) # 0.02920048677256981

In [ ]:
feature_importance2 = rf_run2.feature_importances_
print("Feature Importance:")
col_name2 = []
feature_score2 = []
for feature, importance in zip(X_train2.columns, feature_importance2):
  col_name2.append(feature)
  feature_score2.append(importance)

feature_df2 = pd.DataFrame({'열':col_name2,'feature_importance':feature_score2})
feature_df2.sort_values(by='feature_importance', ascending=False)

In [ ]:
sorted_indices = np.argsort(feature_importance2)[::-1]
sorted_importance = feature_importance2[sorted_indices]
sorted_variables = np.array(X_train2.columns)[sorted_indices]

# 가로막대 그래프 생성
plt.figure(figsize=(8, 6))
plt.barh(sorted_variables, sorted_importance, color='skyblue')
plt.xlabel('중요도')
plt.ylabel('변수 이름')
plt.title('변수 중요도')


### 집단2 회귀계수

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
pd.set_option('display.float_format', '{:.7f}'.format)

# 데이터 불러오기


# for b0, 상수항 추가
x_data21 = sm.add_constant(x_data2[['BOD(㎎/L)_x', 'COD(㎎/L)_x', '총질소(T-N)(㎎/L)', 'I-131(요오드)', 'DO(㎎/L)',
 'TOC(㎎/L)', '방류_BOD', '상구분기별인구수','총_유동인구_수_하구']], has_constant = "add") # 열을 바꾸고 싶다면 sv_per_step[8]이 부분을 바꾸면 됨

# OLS 검정
multi_model2 = sm.OLS(target2, x_data21)
fitted_multi_model2 = multi_model2.fit()
print(fitted_multi_model2.summary())
col_list2 = []
val_list2 = []
variable_names2 = list(fitted_multi_model2.params.index)
coefficients2 = list(fitted_multi_model2.params)
for i, coef in enumerate(coefficients2):
    variable_name = variable_names2[i]
    col_list2.append(variable_name)
    val_list2.append(coef)

coef_df2 = pd.DataFrame({'열':col_list2,'회귀계수':val_list2})
coef_df2.sort_values('회귀계수',ascending=False)


In [ ]:
explainer = shap.TreeExplainer(rf_run1)
shap_values = explainer.shap_values(X_test2)
shap.initjs() # javascript 초기화 (graph 초기화)
shap.force_plot(explainer.expected_value, shap_values[1,:], X_test2.iloc[1,:])
shap.summary_plot ( shap_values , X_test2 )